In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '../..')

#### Data Loading

In [ ]:
# load parameters
import numpy as np
from UltraMotionCapture import field

r = np.load('../../UltraMotionCapture/config/calibrate/r.npy')
s = np.load('../../UltraMotionCapture/config/calibrate/s.npy')
t = np.load('../../UltraMotionCapture/config/calibrate/t.npy')
s, m = field.transform_rst2sm(r, s, t)

In [ ]:
# load mesh
import pyvista as pv
pv.set_jupyter_backend('pythreejs')

mesh = pv.read('../../UltraMotionCapture/data/6kmh_softbra_8markers_1/speed_6km_soft_bra.000001.obj')
texture = pv.read_texture('../../UltraMotionCapture/data/6kmh_softbra_8markers_1/speed_6km_soft_bra.000001.jpg')

mesh.transform(m)
mesh.scale(s)

In [ ]:
mesh.plot(show_edges=True)

#### Fix Dis-connectivity Problem

In [ ]:
# mesh.triangulate()
clean = mesh.clean()
bodies = clean.split_bodies()
# bodies.plot(multi_colors=True, opacity=0.25)

In [ ]:
point_nums = [len(body.points) for body in bodies]
max_index = point_nums.index(max(point_nums))

In [ ]:
surf = bodies[max_index].extract_surface()
surf.plot()

#### Subsampling with `decimate()`

Result shows that `decimate()` evenly sampled the mesh.

In [ ]:
subsample = surf.decimate(0.95)
subsample.plot(show_edges=True)

In [ ]:
import numpy as np
from UltraMotionCapture import obj3d

points = np.array(subsample.points)
pcd = obj3d.np2pcd(points)

In [ ]:
pvpcd = obj3d.np2pvpcd(points)
type(pcd)

In [ ]:
scene = pv.Plotter()
scene.add_points(points)
scene.show()

#### Subsampling with `decimate_pro()`

Result shows that `decimate_pro()` samples the complex structure with denser sampling points, which might be useful for proper alignment since that it can serve as a attention mechanism. However, it might be more unstable. 

In [ ]:
subsample_pro = surf.decimate_pro(0.95)
subsample_pro.plot(show_edges=True, color='white')

In [ ]:
points_pro = np.array(subsample_pro.points)
pcd_pro = obj3d.np2pcd(points_pro)

In [ ]:
scene = pv.Plotter()
scene.add_points(points_pro)
scene.show()

In [ ]:
scene = pv.Plotter()
scene.add_points(points_pro)
scene.add_mesh(subsample_pro.translate((700, 0, 0)), show_edges=True)
scene.show()